In [ ]:
!pip -q install google-generativeai==0.3.0
!pip -q install google-ai-generativelanguage==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.9/145.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
import numpy as np # linear algebra
# !pip install datasets --quiet
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import seaborn as sns

In [ ]:
import pandas as pd
df = pd.read_csv('/content/cwi_training.txt', delimiter = "\t", header = None)
df.columns = ["sentence", "token", "index", "score"]

In [ ]:
df

,sentence,token,index,score
0,In 1832 his family emigrated thence to Bellevi...,thence,5,1
1,In 1832 his family emigrated thence to Bellevi...,apprenticed,13,1
2,In 1832 his family emigrated thence to Bellevi...,family,3,0
3,In 1832 his family emigrated thence to Bellevi...,emigrated,4,0
4,In 1832 his family emigrated thence to Bellevi...,printer,16,0
...,...,...,...,...
2232,The short words math or maths are often used f...,basic,14,0
2233,The short words math or maths are often used f...,algebra,15,1
2234,The short words math or maths are often used f...,young,17,0
2235,The short words math or maths are often used f...,students,18,0


In [ ]:
df["score"].value_counts()

score
0    1531
1     706
Name: count, dtype: int64

In [ ]:
from google.colab import userdata

In [ ]:
# setup
import google.generativeai as genai
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown

GOOGLE_AI_STUDIO = userdata.get('key')

genai.configure(api_key=GOOGLE_AI_STUDIO)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score

def g_score(labels, predicted):

  accuracy = accuracy_score(labels, predicted)
  recall = recall_score(labels, predicted)

  return (2 * accuracy * recall) / (accuracy + recall)

In [ ]:
for m in genai.list_models():
    print(m.name)
    print(m.supported_generation_methods)

models/chat-bison-001
['generateMessage', 'countMessageTokens']
models/text-bison-001
['generateText', 'countTextTokens', 'createTunedTextModel']
models/embedding-gecko-001
['embedText', 'countTextTokens']
models/gemini-1.0-pro
['generateContent', 'countTokens']
models/gemini-1.0-pro-001
['generateContent', 'countTokens', 'createTunedModel']
models/gemini-1.0-pro-latest
['generateContent', 'countTokens']
models/gemini-1.0-pro-vision-latest
['generateContent', 'countTokens']
models/gemini-1.5-flash
['generateContent', 'countTokens']
models/gemini-1.5-flash-001
['generateContent', 'countTokens']
models/gemini-1.5-flash-latest
['generateContent', 'countTokens']
models/gemini-1.5-pro
['generateContent', 'countTokens']
models/gemini-1.5-pro-001
['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro-latest
['generateContent', 'countTokens']
models/gemini-pro
['generateContent', 'countTokens']
models/gemini-pro-vision
['generateContent', 'countTokens']
models/embeddin

In [ ]:
# Set up the model
generation_config = {
  "temperature": 0.0,
  "top_p": 0.6,
  "top_k": 1,
  "max_output_tokens": 100,
}


safe = [
        {
            "category": "HARM_CATEGORY_DANGEROUS",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_NONE",
        },
    ]

from tqdm import tqdm

model = genai.GenerativeModel(model_name="gemini-1.0-pro-latest",
                              generation_config=generation_config, safety_settings = safe)

In [ ]:
import time

df = df.iloc[:35]
result = []
labels = df.score.values

for index, row in tqdm(df.iterrows()):
    prompt = f"""
    Question: Considering that even common words can be complex in certain contexts, is the word '{row['token']}' in the following sentence considered complex for a non-native English speaker? Please answer with 'Yes' or 'No'.
    Context: '{row['sentence']}'
    Note: A word is deemed complex if at least one out of twenty annotators has classified it as such.
    """
    response = model.generate_content(prompt)
    result.append(response.text)
    time.sleep(30)

35it [18:40, 32.02s/it]


In [ ]:
results = [1 if i.rstrip() == "Yes" else 0 for i in result]

In [ ]:
# pro
g_score(labels, results) * 100

83.33333333333333

In [ ]:
recall_score(labels, results) * 100

100.0

In [ ]:
import time


# Set up the model
generation_config = {
  "temperature": 0.0,
  "top_p": 0.6,
  "top_k": 1,
  "max_output_tokens": 100,
}


safe = [
        {
            "category": "HARM_CATEGORY_DANGEROUS",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_NONE",
        },
    ]

from tqdm import tqdm

model = genai.GenerativeModel(model_name="gemini-1.5-flash-latest",
                              generation_config=generation_config, safety_settings = safe)

df = df.iloc[-50:]
result = []
labels = df.score.values

for index, row in tqdm(df.iterrows()):
    prompt = f"""
    Question: Is the word '{row['token']}' in the following sentence considered complex for a non-native English speaker? Please answer with 'Yes' or 'No', Considering that even common words can be complex in certain contexts, .
    Context: '{row['sentence']}'
    Note: A word is deemed complex if at least one out of twenty annotators has classified it as such.
    """
    response = model.generate_content(prompt)
    result.append(response.text)
    time.sleep(10)

50it [09:47, 11.75s/it]


In [ ]:
results = [1 if i.rstrip() == "Yes" else 0 for i in result]

In [ ]:
# flash
g_score(labels, results) * 100

69.76303317535545

In [ ]:
import time


# Set up the model
generation_config = {
  "temperature": 0.0,
  "top_p": 0.6,
  "top_k": 1,
  "max_output_tokens": 100,
}


safe = [
        {
            "category": "HARM_CATEGORY_DANGEROUS",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_NONE",
        },
    ]

from tqdm import tqdm

model = genai.GenerativeModel(model_name="gemini-1.0-pro-latest",
                              generation_config=generation_config, safety_settings = safe)

df = df.iloc[-50:]
result = []
labels = df.score.values

for index, row in tqdm(df.iterrows()):
    prompt = f"""
    ## Complex Word Assessment for Non-Native Speakers

    You are a language expert tasked with identifying potentially complex words for English learners.

    **Word:** {row['token']}
    **Sentence:** "{row['sentence']}"

    **Instructions:**

    * Imagine a learner with an intermediate level of English proficiency.
    * Consider if the word, in its specific context within the sentence, might pose difficulty for this learner.
    * Complexity can stem from:
        * Uncommon vocabulary
        * Figurative language or idioms
        * Technical or subject-specific jargon
        * Nuanced meanings or multiple definitions

    **Respond with "Yes" if the word is likely complex for the learner, and "No" otherwise.**
    """
    response = model.generate_content(prompt)
    result.append(response.text)
    time.sleep(10)

50it [09:38, 11.57s/it]


In [ ]:
results = [1 if i.rstrip() == "Yes" else 0 for i in result]

In [ ]:
# 1.0 pro
print("accuracy", accuracy_score(labels, results) * 100)
print("recall", recall_score(labels, results) * 100)
print("G-Score", g_score(labels, results) * 100)

accuracy 57.99999999999999
recall 87.5
G-Score 69.75945017182129


In [ ]:
pd.set_option('display.max_colwidth', None)

df.iloc[745:750][['sentence', 'token', 'score']]

,sentence,token,score
745,"This consciousness is not merely that of the body or mind and intellect , but of a soul-state above the mind that exists within and beyond our existence , the pure Self of all .",merely,1
746,"This consciousness is not merely that of the body or mind and intellect , but of a soul-state above the mind that exists within and beyond our existence , the pure Self of all .",consciousness,1
747,"This consciousness is not merely that of the body or mind and intellect , but of a soul-state above the mind that exists within and beyond our existence , the pure Self of all .",is,0
748,"This consciousness is not merely that of the body or mind and intellect , but of a soul-state above the mind that exists within and beyond our existence , the pure Self of all .",not,0
749,"This consciousness is not merely that of the body or mind and intellect , but of a soul-state above the mind that exists within and beyond our existence , the pure Self of all .",body,0


In [ ]:
import time


# Set up the model
generation_config = {
  "temperature": 0.0,
  "top_p": 0.6,
  "top_k": 1,
  "max_output_tokens": 100,
}


safe = [
        {
            "category": "HARM_CATEGORY_DANGEROUS",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_NONE",
        },
    ]

from tqdm import tqdm

model = genai.GenerativeModel(model_name="gemini-1.0-pro-latest",
                              generation_config=generation_config, safety_settings = safe)

df = df.iloc[-50:]
result = []
labels = df.score.values

for index, row in tqdm(df.iterrows()):
    prompt = f"""
    Question: Based on the context provided, is the word '{row['token']}' in the following sentence considered complex for a non-native English speaker? Please answer with 'Yes' or 'No'.

    Context: '{row['sentence']}'

    Note: A word is deemed complex if at least one out of twenty annotators has classified it as such. Remember that even commonly used words can be deemed complex depending on their contextual usage.

    Learn from the following examples and use all features you need to accurately answer:
    1. Sentence: 'This consciousness is not merely that of the body or mind and intellect, but of a soul-state above the mind that exists within and beyond our existence, the pure Self of all.'
       Word: 'merely'
       Complexity: Yes

    2. Sentence: 'This consciousness is not merely that of the body or mind and intellect, but of a soul-state above the mind that exists within and beyond our existence, the pure Self of all.'
       Word: 'consciousness'
       Complexity: Yes

    3. Sentence: 'This consciousness is not merely that of the body or mind and intellect, but of a soul-state above the mind that exists within and beyond our existence, the pure Self of all.'
       Word: 'is'
       Complexity: No

    4. Sentence: 'This consciousness is not merely that of the body or mind and intellect, but of a soul-state above the mind that exists within and beyond our existence, the pure Self of all.'
       Word: 'not'
       Complexity: No

    5. Sentence: 'This consciousness is not merely that of the body or mind and intellect, but of a soul-state above the mind that exists within and beyond our existence, the pure Self of all.'
       Word: 'body'4
       Complexity: No
    """

    response = model.generate_content(prompt)
    result.append(response.text)
    time.sleep(10)

50it [09:51, 11.83s/it]


In [ ]:
results = [1 if i.rstrip() == "Yes" else 0 for i in result]

In [ ]:
# 1.0 pro
print("accuracy", accuracy_score(labels, results) * 100)
print("recall", recall_score(labels, results) * 100)
print("G-Score", g_score(labels, results) * 100)

accuracy 86.0
recall 56.25
G-Score 68.01405975395431
